In [ ]:
wildcard = ""
input_nb = ""
output_nb = ""
page_type = ""
analysis_version = "dev"

In [2]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/common/scripts/setup.py
%run -i {root}/workflow/site/scripts/page-setup.py

In [ ]:
def add_header_and_remove_tag(input_nb, output_nb, header, page_type, cell=3):
    with open(input_nb, mode="r", encoding="utf-8") as f:
        my_nb = json.loads(f.read())

    if not page_type == "homepage":
        my_nb["cells"][cell]["source"] = header

    my_nb["cells"][1]["metadata"]["tags"] = ["remove-input"]

    with open(output_nb, "w") as f:
        json.dump(my_nb, f)

## Add headers

In [6]:
gpd_df = gpd.read_file(final_cohorts_geojson_file)

if page_type == "country":
    df = gpd_df.query(f"country_alpha2 == '{wildcard}'")
    country_name = df["country"].unique()[0]
    title = country_name
elif page_type == "contig":
    title = f"Chromosome {wildcard}"
elif page_type == "cohort":
    df = gpd_df.query(f"cohort_id == '{wildcard}'")
    cohort_label = df["cohort_label"].unique()[0]
    title = cohort_label
elif page_type == "homepage":
    title = ""
elif page_type == "alert":
    alert_path = here() / f"workflow/site/alerts/{wildcard}.yaml"
    with open(alert_path, "r") as alert_file:
        alert_config = yaml.safe_load(alert_file)
        alert_id = wildcard
        alert_label = alert_config["label"]
    title = f"Alert `{alert_id}` ({alert_label})"

In [ ]:
add_header_and_remove_tag(
    input_nb=input_nb,
    output_nb=output_nb,
    header=f"# {title}",
    page_type=page_type,
)